In [36]:
import numpy as np
from nnfs.datasets import spiral_data
nnfs.init()

class Layer_Dense:
    def __init__(self, n_inputs, n_neurons, learning_rate=0.01):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        self.learning_rate = learning_rate

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)
        self.weights -= self.learning_rate * self.dweights
        self.biases -= self.learning_rate * self.dbiases

In [37]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

In [38]:
class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)

    def backward(self, dvalues):
        self.dinputs = np.empty_like(dvalues)
        for i, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
            single_output = single_output.reshape(-1, 1)
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            self.dinputs[i] = np.dot(jacobian_matrix, single_dvalues)

In [39]:

class Loss_CategoricalCrossEntropy:
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = len(dvalues[0])

        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


In [40]:
# Hyperparameters
samples = 300
classes = 3
epochs = 500
learning_rate = 0.01

#Generate Data
X, y = spiral_data(samples=samples, classes=classes)


In [41]:
#Initialize Layers
dense1 = Layer_Dense(2, 128, learning_rate)
activation1 = Activation_ReLU()

dense2 = Layer_Dense(128, classes, learning_rate)
activation2 = Activation_Softmax()

loss_function = Loss_CategoricalCrossEntropy()

In [42]:
# Training Loop
for epoch in range(epochs):
    # Forward pass
    dense1.forward(X)
    activation1.forward(dense1.output)
    
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    # Loss calculation
    loss = np.mean(loss_function.forward(activation2.output, y))
    
    predictions = np.argmax(activation2.output, axis=1)
    accuracy = np.mean(predictions == y)

    # Backward pass
    loss_function.backward(activation2.output, y)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)

    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    if epoch % 50 == 0 or epoch == epochs - 1:
        print(f"Epoch: {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

print("\nFinal Accuracy:", accuracy)

Epoch: 0, Loss: 1.0986, Accuracy: 0.2956
Epoch: 50, Loss: 1.0986, Accuracy: 0.3944
Epoch: 100, Loss: 1.0986, Accuracy: 0.4022
Epoch: 150, Loss: 1.0986, Accuracy: 0.4233
Epoch: 200, Loss: 1.0986, Accuracy: 0.4267
Epoch: 250, Loss: 1.0985, Accuracy: 0.4200
Epoch: 300, Loss: 1.0985, Accuracy: 0.4267
Epoch: 350, Loss: 1.0985, Accuracy: 0.4389
Epoch: 400, Loss: 1.0985, Accuracy: 0.4356
Epoch: 450, Loss: 1.0985, Accuracy: 0.4367
Epoch: 499, Loss: 1.0985, Accuracy: 0.4367

Final Accuracy: 0.43666666666666665
